In [1]:
import re

In [7]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [8]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):return True
    else:
        if pattern[0] != saying[0]:return False
        else:
            return pat_match(pattern[1:],saying[1:])
        

In [99]:
'I want ?X'.split()

['I', 'want', '?X']

In [9]:
pat_match("I want ?X".split(), "I want holiday".split())

True

In [10]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [11]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

In [12]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [13]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [14]:
pat_match(pattern, saying)

('?X', 'holiday')

In [15]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

In [16]:
def pat_match(pattern, saying):
    if not pattern or not saying:return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [17]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

[('?X', '2+2'), ('?X', '2+2')]

In [18]:
pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [19]:
def pat_to_dict(patterns):
    return {k:v for k,v in patterns}

In [20]:
def substitute(rule, parsed_rules):
    if not rule:
        return []
    return [parsed_rules.get(rule[0],rule[0])] + substitute(rule[1:], parsed_rules)

In [21]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())

In [22]:
got_patterns

[('?X', 'iPhone')]

In [24]:
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [25]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())
john_pat

[('?P', 'John'), ('?X', 'resting')]

In [26]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [27]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [28]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(john_pat)))

'What if you mean if you got a vacation'

In [29]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [30]:
def get_response(saying, rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    pass

In [64]:
import random

def get_response(saying, rules):
    respose_list = [] # 匹配到多个怎么办，随机返回一个？？
    for  q,a in rules.items():
        parse_result = pat_match(q.split(),saying.split())
        print(f"parse_result:{parse_result}")
        if parse_result:
            respose_sentence = random.choice(a)
            print(f"respose_sentence:{respose_sentence}")
        
            respose = ' '.join(substitute(respose_sentence.split(), pat_to_dict(parse_result)))
            print(f"respose:{respose}")
            respose_list.append(respose)
    return respose_list

In [65]:
' '.join(substitute("Why do you need ?X".split(), pat_to_dict([('?X', 'iPhone')])))

'Why do you need iPhone'

In [66]:
get_response('I need iPhone',defined_patterns)

parse_result:[('?X', 'iPhone')]
respose_sentence:Why do you need ?X ?
respose:Why do you need iPhone ?
parse_result:[]


['Why do you need iPhone ?']

In [68]:
get_response("My mother told me something",defined_patterns)

parse_result:[]
parse_result:[('?X', 'mother')]
respose_sentence:How do you think about your ?X ?
respose:How do you think about your mother ?


['How do you think about your mother ?']

Segment Match

In [71]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*P') and all(a.isalpha()for a in pattern[2:]
                                           )

In [72]:
is_pattern_segment('?*P')

True

In [74]:
from collections import defaultdict

In [87]:

def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i
    
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [88]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [97]:

fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    print("pat = pattern[0] ",pat)
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

In [98]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

pat = pattern[0]  ?*P
pat = pattern[0]  is
pat = pattern[0]  very
pat = pattern[0]  good
pat = pattern[0]  and
pat = pattern[0]  ?*X


[('?P', ['My', 'dog']), True, None]